<a href="https://colab.research.google.com/github/girinath18/AI_Headshot_RG/blob/main/FlaskOnColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask_ngrok
!pip install flask_ngrok
!pip install flask
!pip install pyngrok --upgrade

In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

In [ ]:
!ls

sample_data


In [6]:
!pip install pyngrok
#!ngrok authtoken YOUR_NGROK_AUTH_TOKEN


In [7]:
!ngrok config add-authtoken 2gbCNOE5wJ9vXAdZQQRDAdCP0EU_47uYqH7uXrjbn7pkTsd7X

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 https://ngrok-agent.s3.amazonaws.com buster/main amd64 Packages [4,219 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,848 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,118 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.la

In [ ]:
import os
from flask import Flask, request, jsonify
import logging

app = Flask(__name__)

# Setup logging
logging.basicConfig(level=logging.DEBUG)

def clone_repository(repo_url, target_directory):
    if os.path.exists(target_directory) and os.listdir(target_directory):
        app.logger.info(f"Target directory '{target_directory}' already exists and is not empty.")
    else:
        try:
            os.system(f'git clone {repo_url} {target_directory}')
            app.logger.info("Repository cloned successfully.")
        except Exception as e:
            app.logger.error("Error while cloning repository:", e)

def process_images(source_image_path, output_folder):
    templates_folder_path = r"/content/template"
    os.makedirs(output_folder, exist_ok=True)

    if not os.path.exists(templates_folder_path):
        app.logger.error("Templates folder not found.")
        return "Templates folder not found.", 500

    selected_source_image_path = source_image_path

    for target_image_name in os.listdir(templates_folder_path):
        target_image_path = os.path.join(templates_folder_path, target_image_name)

        if os.path.isfile(target_image_path):
            app.logger.info(f"Processing target image: {target_image_name}")
            os.system(f'python /content/roop/run.py -s "{selected_source_image_path}" -t "{target_image_path}" -o "{output_folder}" --keep-fps --execution-threads 14 --many-faces --execution-provider cuda --frame-processor face_swapper face_enhancer --output-video-quality 35 --temp-frame-format jpg --max-memory 46')

@app.route('/face_swap', methods=['POST'])
def face_swap():
    if 'file' not in request.files:
        app.logger.error("No file part in the request.")
        return jsonify({'error': 'No file part'})

    file = request.files['file']
    if file.filename == '':
        app.logger.error("No selected file.")
        return jsonify({'error': 'No selected file'})

    source_image_path = r"/content/uploaded"
    file.save(source_image_path)

    output_folder = r"/content/output"
    result = process_images(source_image_path, output_folder)
    if isinstance(result, tuple) and result[1] == 500:
        app.logger.error(f"Error during image processing: {result[0]}")
        return jsonify({'error': result[0]}), 500

    output_image_path = os.path.join(output_folder, "output.jpg")
    return jsonify({'output_image_path': output_image_path})

@app.route('/new_endpoint', methods=['POST'])
def new_endpoint():
    return jsonify({'message': 'This is the response from the new endpoint'})

if __name__ == '__main__':
    repo_url = "https://github.com/girinath18/roop.git"
    target_directory = r"/content/roop"
    clone_repository(repo_url, target_directory)
    os.chdir(target_directory)

    # Run the Flask app
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on /face_swap [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-17-83aac882a676>", line 49, in face_swap
    file.save(source_image_path)
  File "/usr/local/lib/python3.10/dist-packages

In [16]:
# Install pyngrok if not already installed
!pip install pyngrok

# Authenticate ngrok with your authtoken
!ngrok authtoken 2gbCNOE5wJ9vXAdZQQRDAdCP0EU_47uYqH7uXrjbn7pkTsd7X

# Start ngrok and expose the Flask app running on port 5000
from pyngrok import ngrok

# Open a tunnel on port 5000 for HTTP traffic
public_url = ngrok.connect(5000)
print(f"ngrok tunnel URL: {public_url}")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
ngrok tunnel URL: NgrokTunnel: "https://a4e1-34-82-193-196.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
import os
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify

app = Flask(__name__)

run_with_ngrok(app)
@app.route('/')
def home():
    return "Hello, Flask is running!"

if __name__ == '__main__':
    app.run(port=5000)


In [ ]:
# Install pyngrok if not already installed
!pip install pyngrok

# Authenticate ngrok with your authtoken
!ngrok authtoken 2gbCNOE5wJ9vXAdZQQRDAdCP0EU_47uYqH7uXrjbn7pkTsd7X

# Start ngrok and expose the Flask app running on port 5000
from pyngrok import ngrok

# Open a tunnel on port 5000 for HTTP traffic
public_url = ngrok.connect(5000)
print(f"ngrok tunnel URL: {public_url}")


# **Sample Code**

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask

app = Flask(__name__)

run_with_ngrok(app)
@app.route("/")
def home():
    return f"Running Flask on Google Colab!"

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6258-35-247-6-25.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Dec/2021 06:21:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 06:21:54] "GET /favicon.ico HTTP/1.1" 404 -
